In [2]:
print("Medical Chatbot")

Medical Chatbot


In [3]:
from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain_pinecone import PineconeVectorStore
import pinecone
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain_community.vectorstores import FAISS


c:\Users\sathv\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
c:\Users\sathv\anaconda3\envs\mchatbot\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [4]:
PINECONE_API_KEY="pcsk_6Lgu1z_4pAz6zZCrFsnHGq6HaSksKJXcjqSSs9pH1psJPrXppgQ6zhXv4gwMP8yGy7oKn8"
PINECONE_API_ENV="us-east-1"

In [5]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extracted_data=load_pdf("data/")

In [7]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks


In [8]:
text_chunks=text_split(extracted_data)



In [9]:
def download_huggingface_embedding():
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [11]:
embeddings=download_huggingface_embedding()
vectorstore=FAISS.from_documents(text_chunks,embeddings)
# model = SentenceTransformer('all-MiniLM-L6-v2')
# print("Loaded:", model)


c:\Users\sathv\anaconda3\envs\mchatbot\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [12]:
query_result=embeddings.embed_query("Hello world")

In [13]:
import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

# Create the index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # or the dimension of your embeddings
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        )
    )

# Connect to the index
index = pc.Index(index_name)
os.environ["PINECONE_API_KEY"]="pcsk_6Lgu1z_4pAz6zZCrFsnHGq6HaSksKJXcjqSSs9pH1psJPrXppgQ6zhXv4gwMP8yGy7oKn8"
# docsearch=LangchainPinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)
docsearch = PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks],
    embeddings,
    index_name=index_name,
    # namespace="default", 
    pinecone_api_key=PINECONE_API_KEY
)


In [14]:
# docsearch=LangchainPinecone.from_existing_index(index,embeddings)
query="What are Allergies"
docs=docsearch.similarity_search(query,k=3)
print(docs)

[Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.\nKEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction—An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva—The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.'), Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.\nKEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction—An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva—The mucous membrane that covers\nthe white part of the eyes and lines the eyeli

In [15]:
prompt_template="""
Use the following pieces of information to answer user's questions
If you don't know the answer, just say that you don't know, don't try to make up an answer
Context:{context}
Question:{question}
Only return helpful answers and nothing else
Helpful answer:
"""

In [16]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":prompt}

In [17]:
llm=CTransformers(model="model/llama-2-13b-chat.ggmlv3.q6_K.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})


In [18]:
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings  # Or your embedding
from langchain_community.llms import LlamaCpp  # Or whatever LLM you are using

# Assuming `docsearch` is your Pinecone vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define chain_type_kwargs only if needed (like for prompt template)
chain_type_kwargs = {}  # or your actual kwargs

# Create the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,    
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)



In [ ]:
while True:
    user_input=input(f"Input prompt:")
    result=qa({"query":user_input})
    print("Response:",result["result"])

C:\Users\sathv\AppData\Local\Temp\ipykernel_37668\2008814045.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result=qa({"query":user_input})


Response:  Acne is a chronic inflammatory disease of the pilosebaceous gland.


Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
